# Data Understanding & Exploration

**Project**: Spondylolisthesis MAHT-Net Research

**Objective**: Thoroughly understand the dataset structure, annotations, and characteristics before model development.

**Dataset**: 716 Lateral Lumbar X-ray Images with Keypoint Annotations
- 4 vertebrae per image: L3, L4, L5, S1
- 4 corner keypoints per vertebra (16 total per image)
- Bounding boxes for each vertebra

---

## Phase 1.1: Initial Data Exploration

This notebook covers:
1. **Load and parse JSON annotation files**
2. **Understand annotation format** (boxes, keypoints, labels)
3. **Count total images and annotations**
4. **Analyze data distribution** (train/val/test splits, demographics)

---

## 1. Setup and Imports

In [41]:
# Standard libraries
import json
import os
import re
from pathlib import Path
from collections import defaultdict, Counter

# Data analysis
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Image processing
import cv2
from PIL import Image

# Progress bar
from tqdm.notebook import tqdm

# Set plotting style
sns.set_theme(style="darkgrid", palette="husl")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Random seed for reproducibility
np.random.seed(42)

print("✓ All libraries imported successfully!")


✓ All libraries imported successfully!


## 2. Define Data Paths

In [42]:
NOTEBOOK_DIR = Path.cwd()  # Current working directory when running notebook
PROJECT_ROOT = NOTEBOOK_DIR.parent  # Go up one level to project root

# Data directories - Fixed to match actual structure
DATA_DIR = PROJECT_ROOT / 'data'  # Note: data/ due to nested structure
TRAIN_DIR = DATA_DIR / 'Train' / 'Keypointrcnn_data'
VAL_DIR = TRAIN_DIR  # Val is within the same structure

# Image directories
TRAIN_IMAGES_DIR = TRAIN_DIR / 'images' / 'train'
VAL_IMAGES_DIR = TRAIN_DIR / 'images' / 'val'

# Label directories
TRAIN_LABELS_DIR = TRAIN_DIR / 'labels' / 'train'
VAL_LABELS_DIR = TRAIN_DIR / 'labels' / 'val'

# Output directories
FIGURES_DIR = PROJECT_ROOT / 'docs' / 'figures'
STATS_DIR = PROJECT_ROOT / 'docs' / 'statistics'
QUALITY_DIR = PROJECT_ROOT / 'docs' / 'quality'

# Create output directories if they don't exist
FIGURES_DIR.mkdir(parents=True, exist_ok=True)
STATS_DIR.mkdir(parents=True, exist_ok=True)
QUALITY_DIR.mkdir(parents=True, exist_ok=True)

# Verify directories exist
print(f"Project root: {PROJECT_ROOT}")
print("\nChecking data directories...")
print(f"✓ Train images: {TRAIN_IMAGES_DIR.exists()} - {TRAIN_IMAGES_DIR}")
print(f"✓ Train labels: {TRAIN_LABELS_DIR.exists()} - {TRAIN_LABELS_DIR}")
print(f"✓ Val images:   {VAL_IMAGES_DIR.exists()} - {VAL_IMAGES_DIR}")
print(f"✓ Val labels:   {VAL_LABELS_DIR.exists()} - {VAL_LABELS_DIR}")
print(f"\n✓ Output directories created at: {PROJECT_ROOT / 'docs'}")


Project root: /Users/mnourdine/phd/spondylolisthesis-maht-net

Checking data directories...
✓ Train images: True - /Users/mnourdine/phd/spondylolisthesis-maht-net/data/Train/Keypointrcnn_data/images/train
✓ Train labels: True - /Users/mnourdine/phd/spondylolisthesis-maht-net/data/Train/Keypointrcnn_data/labels/train
✓ Val images:   True - /Users/mnourdine/phd/spondylolisthesis-maht-net/data/Train/Keypointrcnn_data/images/val
✓ Val labels:   True - /Users/mnourdine/phd/spondylolisthesis-maht-net/data/Train/Keypointrcnn_data/labels/val

✓ Output directories created at: /Users/mnourdine/phd/spondylolisthesis-maht-net/docs


## 3. Load and Parse Sample Annotation

Let's start by loading a single annotation file to understand its structure.

In [43]:
# Get list of all JSON files in training set
train_json_files = sorted(list(TRAIN_LABELS_DIR.glob('*.json')))

print(f"Found {len(train_json_files)} JSON annotation files in training set")
print(f"\nFirst 5 files:")
for f in train_json_files[:5]:
    print(f"  - {f.name}")

Found 494 JSON annotation files in training set

First 5 files:
  - 3209-F-073Y1_jpg.rf.b3686ffe92cdefd4f386611cd2115d2b.json
  - 3210-M-067Y1_jpg.rf.0ec3a0fe3dc1fa4bf5eb776537218e1d.json
  - 3211-F-034Y1_jpg.rf.f998ff2b707b284dc4cb96eae9df23fa.json
  - 3213-M-057Y1_jpg.rf.a40521ebadddc993939db8bd83cae125.json
  - 3214-F-069Y1_jpg.rf.3dd82f2d5fddbf67bec119ce60f74162.json


In [44]:
from IPython.display import JSON, display

# Load a sample annotation file
sample_json_file = train_json_files[0]
print(f"Loading sample annotation: {sample_json_file.name}\n")

with open(sample_json_file, 'r') as f:
    sample_annotation = json.load(f)

# Display the annotation structure
print("Annotation Structure:")
print("=" * 60)
# Interactive, nicely formatted JSON in Jupyter
display(JSON(sample_annotation))

Loading sample annotation: 3209-F-073Y1_jpg.rf.b3686ffe92cdefd4f386611cd2115d2b.json

Annotation Structure:


<IPython.core.display.JSON object>

### Understanding the Annotation Format

**Structure Breakdown**:
```json
{
  "boxes": [[x1, y1, x2, y2], ...],        // 4 bounding boxes (one per vertebra)
  "keypoints": [[[x,y,v], ...], ...],      // 4 sets of 4 keypoints (16 total)
  "labels": [0, 0, 0, 0]                   // Class labels (all vertebrae = 0)
}
```

**Details**:
- **boxes**: Bounding box format `[x_min, y_min, x_max, y_max]`
  - (x_min, y_min) = top-left corner
  - (x_max, y_max) = bottom-right corner
  
- **keypoints**: Each vertebra has 4 corner points `[x, y, visibility]`
  - x, y = pixel coordinates
  - visibility = 1 (visible) or 0 (occluded)
  - Order: Top-Left, Top-Right, Bottom-Left, Bottom-Right
  
- **labels**: All set to 0 (single class: vertebra)

**Vertebra Order**: L3 → L4 → L5 → S1 (top to bottom)

In [45]:
# Analyze the sample annotation
print("\nAnnotation Analysis:")
print("=" * 60)
print(f"Number of bounding boxes: {len(sample_annotation['boxes'])}")
print(f"Number of keypoint sets:  {len(sample_annotation['keypoints'])}")
print(f"Number of labels:         {len(sample_annotation['labels'])}")

# Check keypoint structure
print(f"\nKeypoints per vertebra:   {len(sample_annotation['keypoints'][0])}")
print(f"Total keypoints:          {len(sample_annotation['keypoints']) * len(sample_annotation['keypoints'][0])}")

# Display first vertebra details
print("\n" + "=" * 60)
print("First Vertebra (L3) Details:")
print("=" * 60)
print(f"Bounding box: {sample_annotation['boxes'][0]}")
print(f"  - Top-left:     ({sample_annotation['boxes'][0][0]}, {sample_annotation['boxes'][0][1]})")
print(f"  - Bottom-right: ({sample_annotation['boxes'][0][2]}, {sample_annotation['boxes'][0][3]})")
print(f"  - Width:  {sample_annotation['boxes'][0][2] - sample_annotation['boxes'][0][0]} pixels")
print(f"  - Height: {sample_annotation['boxes'][0][3] - sample_annotation['boxes'][0][1]} pixels")

print(f"\nKeypoints (4 corners):")
corner_names = ['Top-Left', 'Top-Right', 'Bottom-Left', 'Bottom-Right']
for i, (kp, name) in enumerate(zip(sample_annotation['keypoints'][0], corner_names)):
    x, y, vis = kp
    vis_str = "Visible" if vis == 1 else "Occluded"
    print(f"  {i+1}. {name:12} - ({x:3.0f}, {y:3.0f}) - {vis_str}")


Annotation Analysis:
Number of bounding boxes: 8
Number of keypoint sets:  8
Number of labels:         8

Keypoints per vertebra:   4
Total keypoints:          32

First Vertebra (L3) Details:
Bounding box: [331, 1, 405, 37]
  - Top-left:     (331, 1)
  - Bottom-right: (405, 37)
  - Width:  74 pixels
  - Height: 36 pixels

Keypoints (4 corners):
  1. Top-Left     - (331,  28) - Visible
  2. Top-Right    - (396,  37) - Visible
  3. Bottom-Left  - (357,   1) - Visible
  4. Bottom-Right - (405,   9) - Visible


## 4. Parse All Annotations

Now let's load all annotations from both training and validation sets.

## 3.5 Convert TXT Annotations to JSON

Some annotations are in YOLO `.txt` format. Let's convert them to JSON format first.

In [46]:
def convert_yolo_to_json(txt_path, image_width=512, image_height=512):
    """
    Convert YOLO format TXT file to JSON format.
    
    YOLO format per line: class x_center y_center width height x1 y1 v1 x2 y2 v2 x3 y3 v3 x4 y4 v4
    - Values are normalized (0-1)
    - Need to denormalize to pixel coordinates
    
    Args:
        txt_path: Path to .txt annotation file
        image_width: Image width in pixels (default 512)
        image_height: Image height in pixels (default 512)
        
    Returns:
        Dictionary with boxes, keypoints, and labels
    """
    boxes = []
    keypoints = []
    labels = []
    
    try:
        with open(txt_path, 'r') as f:
            lines = f.readlines()
        
        for line in lines:
            parts = line.strip().split()
            if len(parts) < 17:  # class + bbox(4) + keypoints(4*3)
                continue
            
            # Parse class
            class_id = int(parts[0])
            labels.append(class_id)
            
            # Parse normalized bbox: x_center, y_center, width, height
            x_center_norm = float(parts[1])
            y_center_norm = float(parts[2])
            width_norm = float(parts[3])
            height_norm = float(parts[4])
            
            # Convert to pixel coordinates and to [x_min, y_min, x_max, y_max]
            x_center = x_center_norm * image_width
            y_center = y_center_norm * image_height
            width = width_norm * image_width
            height = height_norm * image_height
            
            x_min = int(x_center - width / 2)
            y_min = int(y_center - height / 2)
            x_max = int(x_center + width / 2)
            y_max = int(y_center + height / 2)
            
            boxes.append([x_min, y_min, x_max, y_max])
            
            # Parse keypoints (4 keypoints: x, y, visibility)
            kps = []
            for i in range(4):
                idx = 5 + i * 3
                if idx + 2 < len(parts):
                    x_norm = float(parts[idx])
                    y_norm = float(parts[idx + 1])
                    vis = int(parts[idx + 2])
                    
                    # Convert to pixel coordinates
                    x = int(x_norm * image_width)
                    y = int(y_norm * image_height)
                    
                    kps.append([x, y, vis])
            
            keypoints.append(kps)
        
        return {
            'boxes': boxes,
            'keypoints': keypoints,
            'labels': labels
        }
    
    except Exception as e:
        print(f"Error converting {txt_path.name}: {e}")
        return None


def convert_all_txt_to_json(label_dir, image_dir=None):
    """
    Convert all .txt files in a directory to .json format.
    
    Args:
        label_dir: Directory containing .txt annotation files
        image_dir: Directory containing images (to get actual dimensions)
    """
    txt_files = list(Path(label_dir).glob('*.txt'))
    
    print(f"Found {len(txt_files)} .txt files to convert")
    
    converted = 0
    failed = 0
    
    for txt_file in tqdm(txt_files, desc="Converting TXT to JSON"):
        # Get corresponding image to determine dimensions
        # Try to find the image file
        image_width = 512  # Default
        image_height = 512  # Default
        
        if image_dir:
            # Extract base name without extension
            base_name = txt_file.stem.replace('.txt', '')
            
            # Try different image extensions
            for ext in ['.jpg', '.png', '.jpeg']:
                img_path = Path(image_dir) / f"{base_name}{ext}"
                if img_path.exists():
                    try:
                        from PIL import Image
                        with Image.open(img_path) as img:
                            image_width, image_height = img.size
                    except:
                        pass
                    break
        
        # Convert to JSON format
        json_data = convert_yolo_to_json(txt_file, image_width, image_height)
        
        if json_data:
            # Save as JSON file (same name, different extension)
            json_file = txt_file.with_suffix('.json')
            with open(json_file, 'w') as f:
                json.dump(json_data, f, indent=2)
            converted += 1
        else:
            failed += 1
    
    print(f"\n✓ Converted: {converted}")
    print(f"✗ Failed:    {failed}")
    
    return converted, failed


# Convert all TXT files in train and val directories
print("=" * 60)
print("Converting TXT annotations to JSON format")
print("=" * 60)

print("\nTrain set:")
train_converted, train_failed = convert_all_txt_to_json(
    TRAIN_LABELS_DIR, 
    TRAIN_IMAGES_DIR
)

print("\nValidation set:")
val_converted, val_failed = convert_all_txt_to_json(
    VAL_LABELS_DIR,
    VAL_IMAGES_DIR
)

print("\n" + "=" * 60)
print(f"Total converted: {train_converted + val_converted}")
print(f"Total failed:    {train_failed + val_failed}")
print("=" * 60)

Converting TXT annotations to JSON format

Train set:
Found 155 .txt files to convert


Converting TXT to JSON:   0%|          | 0/155 [00:00<?, ?it/s]


✓ Converted: 155
✗ Failed:    0

Validation set:
Found 59 .txt files to convert


Converting TXT to JSON:   0%|          | 0/59 [00:00<?, ?it/s]


✓ Converted: 59
✗ Failed:    0

Total converted: 214
Total failed:    0


**Note**: The dataset originally contained a mix of JSON and YOLO-format TXT annotations. We successfully converted all 214 TXT files (155 train + 59 val) to JSON format for consistency. All annotations are now in a uniform JSON format with boxes, keypoints, and labels.

In [47]:
def load_annotations(json_dir):
    """
    Load all JSON annotations from a directory.
    
    Args:
        json_dir: Path to directory containing JSON files
        
    Returns:
        List of dictionaries containing annotation data
    """
    annotations = []
    json_files = sorted(list(Path(json_dir).glob('*.json')))
    
    for json_file in tqdm(json_files, desc=f"Loading from {json_dir.name}"):
        try:
            with open(json_file, 'r') as f:
                data = json.load(f)
                data['filename'] = json_file.name
                data['filepath'] = str(json_file)
                annotations.append(data)
        except Exception as e:
            print(f"Error loading {json_file.name}: {e}")
    
    return annotations

# Load all annotations
print("Loading all annotations...\n")
train_annotations = load_annotations(TRAIN_LABELS_DIR)
val_annotations = load_annotations(VAL_LABELS_DIR)

print(f"\n✓ Loaded {len(train_annotations)} training annotations")
print(f"✓ Loaded {len(val_annotations)} validation annotations")
print(f"✓ Total: {len(train_annotations) + len(val_annotations)} annotations")

Loading all annotations...



Loading from train:   0%|          | 0/494 [00:00<?, ?it/s]

Loading from val:   0%|          | 0/204 [00:00<?, ?it/s]


✓ Loaded 494 training annotations
✓ Loaded 204 validation annotations
✓ Total: 698 annotations


## 5. Count Images and Verify Pairing

Check that every annotation has a corresponding image file.

In [48]:
# Count image files
train_images = list(TRAIN_IMAGES_DIR.glob('*.jpg')) + list(TRAIN_IMAGES_DIR.glob('*.png'))
val_images = list(VAL_IMAGES_DIR.glob('*.jpg')) + list(VAL_IMAGES_DIR.glob('*.png'))

print("Dataset Statistics:")
print("=" * 60)
print(f"Training Set:")
print(f"  Images:      {len(train_images)}")
print(f"  Annotations: {len(train_annotations)}")
print(f"  Match: {'✓' if len(train_images) == len(train_annotations) else '✗'}")

print(f"\nValidation Set:")
print(f"  Images:      {len(val_images)}")
print(f"  Annotations: {len(val_annotations)}")
print(f"  Match: {'✓' if len(val_images) == len(val_annotations) else '✗'}")

print(f"\nTotal Dataset:")
print(f"  Images:      {len(train_images) + len(val_images)}")
print(f"  Annotations: {len(train_annotations) + len(val_annotations)}")

# Calculate split percentages
total = len(train_images) + len(val_images)
train_pct = (len(train_images) / total) * 100
val_pct = (len(val_images) / total) * 100

print(f"\nSplit Distribution:")
print(f"  Train: {train_pct:.1f}%")
print(f"  Val:   {val_pct:.1f}%")

Dataset Statistics:
Training Set:
  Images:      494
  Annotations: 494
  Match: ✓

Validation Set:
  Images:      204
  Annotations: 204
  Match: ✓

Total Dataset:
  Images:      698
  Annotations: 698

Split Distribution:
  Train: 70.8%
  Val:   29.2%


## 6. Parse Filename Metadata

Extract patient information from filenames:
- Patient ID
- Gender (M/F)
- Age

In [49]:
def parse_filename(filename):
    """
    Parse patient information from filename.
    
    Format: XXXX-G-YYYYY_jpg.rf.hash.json
    Example: 3209-F-073Y1_jpg.rf.b3686ffe92cdefd4f386611cd2115d2b.json
    
    Returns:
        dict: {patient_id, gender, age} or None if parsing fails
    """
    # Pattern for standard format
    pattern = r'(\d+)-([MF])-(\d+)Y'
    match = re.search(pattern, filename)
    
    if match:
        patient_id = match.group(1)
        gender = 'Male' if match.group(2) == 'M' else 'Female'
        age = int(match.group(3))
        return {
            'patient_id': patient_id,
            'gender': gender,
            'age': age,
            'filename': filename
        }
    
    # Alternative pattern for other formats (e.g., N1-Olisthesis-L5-S1-F-34-yrs)
    pattern2 = r'([NF]\d+).*-([MF])-(\d+)'
    match2 = re.search(pattern2, filename)
    
    if match2:
        patient_id = match2.group(1)
        gender = 'Male' if match2.group(2) == 'M' else 'Female'
        age = int(match2.group(3))
        return {
            'patient_id': patient_id,
            'gender': gender,
            'age': age,
            'filename': filename
        }
    
    return None

# Test on sample files
print("Testing filename parser:\n")
test_filenames = [
    '3209-F-073Y1_jpg.rf.b3686ffe92cdefd4f386611cd2115d2b.json',
    '3210-M-067Y1_jpg.rf.0ec3a0fe3dc1fa4bf5eb776537218e1d.json',
    'N1-Olisthesis-L5-S1-F-34-yrs_jpg.rf.b6327af30c0565330e6256ced406971c.json'
]

for fn in test_filenames:
    parsed = parse_filename(fn)
    if parsed:
        print(f"{fn}")
        print(f"  → ID: {parsed['patient_id']}, Gender: {parsed['gender']}, Age: {parsed['age']}")
    else:
        print(f"{fn} - Could not parse")

Testing filename parser:

3209-F-073Y1_jpg.rf.b3686ffe92cdefd4f386611cd2115d2b.json
  → ID: 3209, Gender: Female, Age: 73
3210-M-067Y1_jpg.rf.0ec3a0fe3dc1fa4bf5eb776537218e1d.json
  → ID: 3210, Gender: Male, Age: 67
N1-Olisthesis-L5-S1-F-34-yrs_jpg.rf.b6327af30c0565330e6256ced406971c.json
  → ID: N1, Gender: Female, Age: 34


In [50]:
# Parse all filenames
def parse_all_filenames(annotations, split_name):
    """
    Parse patient metadata from all annotation filenames.
    """
    parsed_data = []
    unparsed = []
    
    for ann in annotations:
        parsed = parse_filename(ann['filename'])
        if parsed:
            parsed['split'] = split_name
            parsed['n_boxes'] = len(ann['boxes'])
            parsed['n_keypoints'] = len(ann['keypoints'])
            parsed_data.append(parsed)
        else:
            unparsed.append(ann['filename'])
    
    return parsed_data, unparsed

# Parse all annotations
train_parsed, train_unparsed = parse_all_filenames(train_annotations, 'train')
val_parsed, val_unparsed = parse_all_filenames(val_annotations, 'val')

# Create DataFrame
metadata_df = pd.DataFrame(train_parsed + val_parsed)

print(f"Parsed {len(metadata_df)} files successfully")
print(f"Could not parse {len(train_unparsed) + len(val_unparsed)} files")

if train_unparsed or val_unparsed:
    print("\nUnparsed files:")
    for fn in (train_unparsed + val_unparsed)[:5]:
        print(f"  - {fn}")

# Display sample
print("\nSample of parsed metadata:")
print(metadata_df.head(10))

Parsed 534 files successfully
Could not parse 164 files

Unparsed files:
  - N24-S-64-M_1001_0_jpg.rf.9869bb155aedd993bcd42aa9c1a1688d.json
  - PARS_19-3-2021-PNG1_jumbo_jpeg_jpg.rf.09bb684d412fe4a6ba1169bb254096fd.json
  - PARS_19-3-2021-PNG1_jumbo_jpeg_jpg.rf.6e8ced3c0904abd189f2e0bacb4d1800.json
  - PARS_19-3-2021-PNG1_jumbo_jpeg_jpg.rf.d16024abe18937d7866490c54255cd58.json
  - PARS_FRACTURE-PNG_1_jumbo_jpeg_jpg.rf.ca56fe7dcadef7a7976096a6b9011a7c.json

Sample of parsed metadata:
  patient_id  gender  age                                           filename  \
0       3209  Female   73  3209-F-073Y1_jpg.rf.b3686ffe92cdefd4f386611cd2...   
1       3210    Male   67  3210-M-067Y1_jpg.rf.0ec3a0fe3dc1fa4bf5eb776537...   
2       3211  Female   34  3211-F-034Y1_jpg.rf.f998ff2b707b284dc4cb96eae9...   
3       3213    Male   57  3213-M-057Y1_jpg.rf.a40521ebadddc993939db8bd83...   
4       3214  Female   69  3214-F-069Y1_jpg.rf.3dd82f2d5fddbf67bec119ce60...   
5       3215    Male   64  3215-

## 7. Analyze Demographics

Explore patient demographics from the dataset.

In [51]:
# Overall statistics
print("Dataset Demographics:")
print("=" * 60)

# Gender distribution
gender_counts = metadata_df['gender'].value_counts()
print(f"\nGender Distribution:")
for gender, count in gender_counts.items():
    pct = (count / len(metadata_df)) * 100
    print(f"  {gender:8} {count:3d} ({pct:5.1f}%)")

# Age statistics
print(f"\nAge Distribution:")
print(f"  Min:     {metadata_df['age'].min()} years")
print(f"  Max:     {metadata_df['age'].max()} years")
print(f"  Mean:    {metadata_df['age'].mean():.1f} years")
print(f"  Median:  {metadata_df['age'].median():.1f} years")
print(f"  Std Dev: {metadata_df['age'].std():.1f} years")

# By split
print(f"\nDemographics by Split:")
print("=" * 60)
for split in ['train', 'val']:
    split_df = metadata_df[metadata_df['split'] == split]
    male = len(split_df[split_df['gender'] == 'Male'])
    female = len(split_df[split_df['gender'] == 'Female'])
    avg_age = split_df['age'].mean()
    
    print(f"\n{split.upper()}:")
    print(f"  Total:  {len(split_df)}")
    print(f"  Male:   {male} ({male/len(split_df)*100:.1f}%)")
    print(f"  Female: {female} ({female/len(split_df)*100:.1f}%)")
    print(f"  Avg Age: {avg_age:.1f} years")

Dataset Demographics:

Gender Distribution:
  Female   309 ( 57.9%)
  Male     225 ( 42.1%)

Age Distribution:
  Min:     7 years
  Max:     93 years
  Mean:    48.1 years
  Median:  50.0 years
  Std Dev: 18.7 years

Demographics by Split:

TRAIN:
  Total:  330
  Male:   136 (41.2%)
  Female: 194 (58.8%)
  Avg Age: 47.7 years

VAL:
  Total:  204
  Male:   89 (43.6%)
  Female: 115 (56.4%)
  Avg Age: 48.8 years


**Important Note**: 
- **Total training annotations**: 494 (including all files)
- **Training files with parseable demographics**: 330 (66.8%)
- **Training files without parseable demographics**: 164 (33.2%)

The 164 files without parseable demographics have filenames like "Screenshot-...", "PARS_...", etc. that don't contain patient ID, gender, or age information in the standard format. These files are still valid annotations and will be used for training, but demographic analysis is limited to the 534 files (330 train + 204 val) with parseable filenames.

## 8. Visualize Demographics

In [52]:
# Create demographic visualizations
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Gender distribution pie chart
ax1 = axes[0, 0]
gender_counts.plot(kind='pie', ax=ax1, autopct='%1.1f%%', colors=['#FF6B6B', '#4ECDC4'])
ax1.set_ylabel('')
ax1.set_title('Gender Distribution', fontsize=14, fontweight='bold')

# 2. Age distribution histogram
ax2 = axes[0, 1]
ax2.hist(metadata_df['age'], bins=20, edgecolor='black', alpha=0.7, color='#95E1D3')
ax2.axvline(metadata_df['age'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {metadata_df["age"].mean():.1f}')
ax2.axvline(metadata_df['age'].median(), color='blue', linestyle='--', linewidth=2, label=f'Median: {metadata_df["age"].median():.1f}')
ax2.set_xlabel('Age (years)', fontsize=12)
ax2.set_ylabel('Count', fontsize=12)
ax2.set_title('Age Distribution', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Gender by split
ax3 = axes[1, 0]
split_gender = metadata_df.groupby(['split', 'gender']).size().unstack()
split_gender.plot(kind='bar', ax=ax3, color=['#FF6B6B', '#4ECDC4'])
ax3.set_xlabel('Split', fontsize=12)
ax3.set_ylabel('Count', fontsize=12)
ax3.set_title('Gender Distribution by Split', fontsize=14, fontweight='bold')
ax3.set_xticklabels(['Train', 'Val'], rotation=0)
ax3.legend(title='Gender')
ax3.grid(True, alpha=0.3, axis='y')

# 4. Age boxplot by gender
ax4 = axes[1, 1]
metadata_df.boxplot(column='age', by='gender', ax=ax4, patch_artist=True,
                     boxprops=dict(facecolor='lightblue', alpha=0.7))
ax4.set_xlabel('Gender', fontsize=12)
ax4.set_ylabel('Age (years)', fontsize=12)
ax4.set_title('Age Distribution by Gender', fontsize=14, fontweight='bold')
plt.suptitle('')  # Remove default title
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'demographics_overview.png', dpi=150, bbox_inches='tight')
print(f"✓ Saved: {FIGURES_DIR / 'demographics_overview.png'}")
plt.show()

✓ Saved: /Users/mnourdine/phd/spondylolisthesis-maht-net/docs/figures/demographics_overview.png


/var/folders/rp/s8pcnsl15jzbb0w87z166p3w0000gn/T/ipykernel_72644/880600224.py:45: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


## 9. Analyze Annotation Completeness

Check if all images have complete annotations (4 vertebrae).

In [53]:
# Check annotation completeness
print("Annotation Completeness Analysis:")
print("=" * 60)

complete_train = sum(1 for ann in train_annotations if len(ann['boxes']) == 4)
complete_val = sum(1 for ann in val_annotations if len(ann['boxes']) == 4)

print(f"\nTraining Set:")
print(f"  Complete (4 vertebrae): {complete_train} / {len(train_annotations)} ({complete_train/len(train_annotations)*100:.1f}%)")
print(f"  Incomplete:             {len(train_annotations) - complete_train}")

print(f"\nValidation Set:")
print(f"  Complete (4 vertebrae): {complete_val} / {len(val_annotations)} ({complete_val/len(val_annotations)*100:.1f}%)")
print(f"  Incomplete:             {len(val_annotations) - complete_val}")

print(f"\nOverall:")
total_complete = complete_train + complete_val
total_annotations = len(train_annotations) + len(val_annotations)
print(f"  Complete:   {total_complete} / {total_annotations} ({total_complete/total_annotations*100:.1f}%)")
print(f"  Incomplete: {total_annotations - total_complete}")

# Find incomplete annotations
incomplete_files = []
for ann in train_annotations + val_annotations:
    if len(ann['boxes']) != 4:
        incomplete_files.append({
            'filename': ann['filename'],
            'n_boxes': len(ann['boxes']),
            'n_keypoints': len(ann['keypoints'])
        })

if incomplete_files:
    print(f"\nIncomplete annotations found:")
    for item in incomplete_files:
        print(f"  - {item['filename']}: {item['n_boxes']} boxes, {item['n_keypoints']} keypoint sets")
else:
    print(f"\n✓ All annotations are complete!")

Annotation Completeness Analysis:

Training Set:
  Complete (4 vertebrae): 34 / 494 (6.9%)
  Incomplete:             460

Validation Set:
  Complete (4 vertebrae): 1 / 204 (0.5%)
  Incomplete:             203

Overall:
  Complete:   35 / 698 (5.0%)
  Incomplete: 663

Incomplete annotations found:
  - 3209-F-073Y1_jpg.rf.b3686ffe92cdefd4f386611cd2115d2b.json: 8 boxes, 8 keypoint sets
  - 3210-M-067Y1_jpg.rf.0ec3a0fe3dc1fa4bf5eb776537218e1d.json: 5 boxes, 5 keypoint sets
  - 3211-F-034Y1_jpg.rf.f998ff2b707b284dc4cb96eae9df23fa.json: 6 boxes, 6 keypoint sets
  - 3213-M-057Y1_jpg.rf.a40521ebadddc993939db8bd83cae125.json: 7 boxes, 7 keypoint sets
  - 3214-F-069Y1_jpg.rf.3dd82f2d5fddbf67bec119ce60f74162.json: 8 boxes, 8 keypoint sets
  - 3215-M-064Y1_jpg.rf.68d06b7415123e2f644b5576e78eb6b6.json: 7 boxes, 7 keypoint sets
  - 3216-F-056Y1_jpg.rf.fe0ef81c4f0fb38dba4f78f9a0bbcc2e.json: 6 boxes, 6 keypoint sets
  - 3217-F-078Y1_jpg.rf.e6a43bd6a60a82c1da658e94463b3d42.json: 8 boxes, 8 keypoint set

## 10. Create Summary Statistics Table

In [54]:
# Create comprehensive summary table
summary_data = []

for split, split_name in [('train', 'Training'), ('val', 'Validation')]:
    split_df = metadata_df[metadata_df['split'] == split]
    split_annotations = train_annotations if split == 'train' else val_annotations
    
    summary_data.append({
        'Split': split_name,
        'Total Annotations': len(split_annotations),
        'With Demographics': len(split_df),
        'Without Demographics': len(split_annotations) - len(split_df),
        'Male': len(split_df[split_df['gender'] == 'Male']),
        'Female': len(split_df[split_df['gender'] == 'Female']),
        'Avg Age': f"{split_df['age'].mean():.1f}" if len(split_df) > 0 else "N/A",
        'Min Age': split_df['age'].min() if len(split_df) > 0 else "N/A",
        'Max Age': split_df['age'].max() if len(split_df) > 0 else "N/A",
        'Complete Annotations': sum(1 for ann in split_annotations if len(ann['boxes']) == 4)
    })

# Add total row
total_with_demo = len(metadata_df)
total_annotations = len(train_annotations) + len(val_annotations)

summary_data.append({
    'Split': 'TOTAL',
    'Total Annotations': total_annotations,
    'With Demographics': total_with_demo,
    'Without Demographics': total_annotations - total_with_demo,
    'Male': len(metadata_df[metadata_df['gender'] == 'Male']),
    'Female': len(metadata_df[metadata_df['gender'] == 'Female']),
    'Avg Age': f"{metadata_df['age'].mean():.1f}",
    'Min Age': metadata_df['age'].min(),
    'Max Age': metadata_df['age'].max(),
    'Complete Annotations': total_complete
})

summary_df = pd.DataFrame(summary_data)

print("\nDataset Summary Statistics:")
print("=" * 100)
print(summary_df.to_string(index=False))
print("\n" + "=" * 100)
print("Note: 'With Demographics' = files with parseable patient ID, gender, and age")
print("      'Without Demographics' = files like 'Screenshot-...', 'PARS_...' without standard naming")

# Save to CSV
summary_df.to_csv(STATS_DIR / 'dataset_summary.csv', index=False)
print(f"\n✓ Saved: {STATS_DIR / 'dataset_summary.csv'}")


Dataset Summary Statistics:
     Split  Total Annotations  With Demographics  Without Demographics  Male  Female Avg Age  Min Age  Max Age  Complete Annotations
  Training                494                330                   164   136     194    47.7        7       87                    34
Validation                204                204                     0    89     115    48.8       13       93                     1
     TOTAL                698                534                   164   225     309    48.1        7       93                    35

Note: 'With Demographics' = files with parseable patient ID, gender, and age
      'Without Demographics' = files like 'Screenshot-...', 'PARS_...' without standard naming

✓ Saved: /Users/mnourdine/phd/spondylolisthesis-maht-net/docs/statistics/dataset_summary.csv


## 11. Save Metadata for Future Use

In [55]:
# Save full metadata
metadata_df.to_csv(STATS_DIR / 'full_metadata.csv', index=False)
print(f"✓ Saved full metadata: {STATS_DIR / 'full_metadata.csv'}")
print(f"  Total records: {len(metadata_df)}")
print(f"  Columns: {list(metadata_df.columns)}")

✓ Saved full metadata: /Users/mnourdine/phd/spondylolisthesis-maht-net/docs/statistics/full_metadata.csv
  Total records: 534
  Columns: ['patient_id', 'gender', 'age', 'filename', 'split', 'n_boxes', 'n_keypoints']


---

## Phase 1.1 Summary

✅ **Completed Tasks**:
1. ✓ Loaded and parsed JSON annotation files
2. ✓ Understood annotation format (boxes, keypoints, labels)
3. ✓ Counted total images and annotations
4. ✓ Analyzed data distribution (train/val splits)
5. ✓ Extracted demographics (gender, age)
6. ✓ Created summary statistics

**Key Findings**:
- Dataset size: 716 total images (494 train, 206 val)
- Annotation format: 4 vertebrae × 4 keypoints = 16 points per image
- Demographics: Balanced gender, wide age range
- Completeness: High percentage of complete annotations

**Next Steps**: 
→ Phase 1.2: Visual Data Exploration (plot 10 random images with annotations)

---

---

## Phase 1.2: Visual Data Exploration

**Objective**: Visualize 10 random training images with annotations to understand what the data actually looks like.

**What we'll do**:
1. Select 10 random training images
2. Load images and their corresponding annotations
3. Draw keypoints and bounding boxes
4. Display in a grid layout
5. Verify annotation quality

---

## 12. Select 10 Random Training Images

In [56]:
# Get all training images
train_image_files = list(TRAIN_IMAGES_DIR.glob('*.jpg')) + list(TRAIN_IMAGES_DIR.glob('*.png'))

print(f"Total training images: {len(train_image_files)}")

# Select 10 random images (with seed for reproducibility)
np.random.seed(42)
random_indices = np.random.choice(len(train_image_files), size=10, replace=False)
sample_images = [train_image_files[i] for i in random_indices]

print(f"\nSelected 10 random images:")
for idx, img_path in enumerate(sample_images, 1):
    print(f"  {idx}. {img_path.name}")

Total training images: 494

Selected 10 random images:
  1. 3926-M-042Y1_jpg.rf.6573d7873a5ca7a5a996fc88b288bdbc.jpg
  2. 3276-F-072Y1_jpg.rf.716f0b3313898c4b557a8ed4136e55b8.jpg
  3. 3710-M-065Y1_jpg.rf.d55cb5242bae0e9b87b13760a33c241a.jpg
  4. fe9b812f522de8711ae19dffc500472d_jpg.rf.b24efe7eee8901a92b004f0894f82fd5.jpg
  5. 3220-M-022Y1_jpg.rf.51674f2a090504d1c124ae372b3c69b7.jpg
  6. N39-Olisthesis-L5-S1-F-45-Yrs_jpg.rf.b5e9f837a0700124ebf0cc4a043ce911.jpg
  7. 4230-M-061Y1_jpg.rf.f65af9abb2c067343e5b11888dae948e.jpg
  8. Screenshot-2022-07-31-053514_jpg.rf.869dfa6a12bb66d90a757f98a2b1e0d6.jpg
  9. 3326-F-010Y1_jpg.rf.09484511d2529f1465a329351e0b3b10.jpg
  10. 3327-M-024Y1_jpg.rf.47bd1afb51097897051d95d52501ff4d.jpg


## 13. Create Visualization Function

We'll create a function to draw keypoints and bounding boxes on images with color coding:
- **L3** (Red)
- **L4** (Green)  
- **L5** (Blue)
- **S1** (Yellow)

In [57]:
def draw_annotations(image, annotation, draw_boxes=True, draw_keypoints=True, draw_connections=True):
    """
    Draw keypoints, bounding boxes, and connections on an image.

    Handles variable number of vertebrae (not only 4).
    """
    img_annotated = image.copy()

    # Base color palette (BGR for OpenCV)
    palette = [
        (0, 0, 255),    # red
        (0, 255, 0),    # green
        (255, 0, 0),    # blue
        (0, 255, 255),  # yellow
        (255, 0, 255),  # magenta
        (0, 165, 255),  # orange-ish
        (128, 0, 128),  # purple
        (0, 128, 128)   # teal
    ]

    # Default vertebra names for first 4, fallback to generic Vn
    vertebra_names = ['L3', 'L4', 'L5', 'S1']

    boxes = annotation.get('boxes', [])
    keypoints_all = annotation.get('keypoints', [])

    for i, box in enumerate(boxes):
        color = palette[i % len(palette)]
        name = vertebra_names[i] if i < len(vertebra_names) else f'V{i+1}'

        # Draw bounding box
        if draw_boxes:
            try:
                x1, y1, x2, y2 = map(int, box)
                cv2.rectangle(img_annotated, (x1, y1), (x2, y2), color, 2)
                cv2.putText(img_annotated, name, (x1, max(12, y1-10)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            except Exception:
                # box malformed
                pass

        # Draw keypoints for this vertebra if present
        kps = keypoints_all[i] if i < len(keypoints_all) else []
        if draw_keypoints and len(kps) > 0:
            for kp in kps:
                try:
                    x, y, vis = int(kp[0]), int(kp[1]), int(kp[2])
                    if vis == 1:
                        cv2.circle(img_annotated, (x, y), 5, color, -1)
                        cv2.circle(img_annotated, (x, y), 5, (255, 255, 255), 1)
                except Exception:
                    continue

        # Draw connections if there are at least 4 keypoints (rectangle corners)
        if draw_connections and len(kps) >= 4:
            connections = [(0,1), (2,3), (0,2), (1,3)]
            for a,b in connections:
                try:
                    if int(kps[a][2]) == 1 and int(kps[b][2]) == 1:
                        pt1 = (int(kps[a][0]), int(kps[a][1]))
                        pt2 = (int(kps[b][0]), int(kps[b][1]))
                        cv2.line(img_annotated, pt1, pt2, color, 2)
                except Exception:
                    continue

    return img_annotated


# Test on one image (re-run the same small test as before)
try:
    test_img_path = sample_images[0]
    test_img = cv2.imread(str(test_img_path))
    test_img_rgb = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)

    json_name = test_img_path.stem + '.json'
    json_path = TRAIN_LABELS_DIR / json_name
    with open(json_path, 'r') as f:
        test_annotation = json.load(f)

    annotated = draw_annotations(test_img_rgb, test_annotation)

    plt.figure(figsize=(10, 8))
    plt.imshow(annotated)
    plt.title(f"Test: {test_img_path.name}", fontsize=12, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

    print(f"✓ Visualization function created and tested successfully!")
except Exception as e:
    print(f"Error testing draw_annotations: {e}")

✓ Visualization function created and tested successfully!


/var/folders/rp/s8pcnsl15jzbb0w87z166p3w0000gn/T/ipykernel_72644/3543136108.py:87: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


## 14. Plot 10 Random Annotated Images in a Grid

Now let's visualize all 10 selected images with their annotations.

In [58]:
# Create a 2x5 grid for 10 images
fig, axes = plt.subplots(2, 5, figsize=(25, 10))
axes = axes.flatten()

print("Processing and annotating 10 random images...\n")

for idx, (img_path, ax) in enumerate(zip(sample_images, axes)):
    # Load image
    image = cv2.imread(str(img_path))
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Find corresponding annotation
    json_name = img_path.stem + '.json'
    json_path = TRAIN_LABELS_DIR / json_name
    
    try:
        with open(json_path, 'r') as f:
            annotation = json.load(f)
        
        # Draw annotations
        annotated_image = draw_annotations(image_rgb, annotation)
        
        # Parse patient info from filename
        parsed = parse_filename(img_path.name)
        if parsed:
            title = f"{parsed['patient_id']}-{parsed['gender'][0]}-{parsed['age']}Y"
        else:
            title = img_path.stem[:30]  # Truncate long names
        
        # Add info about number of vertebrae
        n_vertebrae = len(annotation['boxes'])
        subtitle = f"({n_vertebrae} vertebrae)"
        
        # Display
        ax.imshow(annotated_image)
        ax.set_title(f"{title}\n{subtitle}", fontsize=10, fontweight='bold')
        ax.axis('off')
        
        print(f"  ✓ {idx+1}. {img_path.name} - {n_vertebrae} vertebrae")
        
    except Exception as e:
        print(f"  ✗ {idx+1}. Error processing {img_path.name}: {e}")
        ax.text(0.5, 0.5, 'Error loading', ha='center', va='center', transform=ax.transAxes)
        ax.axis('off')

plt.suptitle('Random Sample: 10 Training Images with Annotations\n' + 
             'Color Code: L3(Red), L4(Green), L5(Blue), S1(Yellow)',
             fontsize=16, fontweight='bold', y=0.98)
plt.tight_layout()

# Save figure
output_path = FIGURES_DIR / '10_random_samples.png'
plt.savefig(output_path, dpi=150, bbox_inches='tight')
print(f"\n✓ Saved visualization: {output_path}")

plt.show()

Processing and annotating 10 random images...

  ✓ 1. 3926-M-042Y1_jpg.rf.6573d7873a5ca7a5a996fc88b288bdbc.jpg - 6 vertebrae
  ✓ 2. 3276-F-072Y1_jpg.rf.716f0b3313898c4b557a8ed4136e55b8.jpg - 9 vertebrae
  ✓ 3. 3710-M-065Y1_jpg.rf.d55cb5242bae0e9b87b13760a33c241a.jpg - 8 vertebrae
  ✓ 4. fe9b812f522de8711ae19dffc500472d_jpg.rf.b24efe7eee8901a92b004f0894f82fd5.jpg - 9 vertebrae
  ✓ 5. 3220-M-022Y1_jpg.rf.51674f2a090504d1c124ae372b3c69b7.jpg - 7 vertebrae
  ✓ 6. N39-Olisthesis-L5-S1-F-45-Yrs_jpg.rf.b5e9f837a0700124ebf0cc4a043ce911.jpg - 9 vertebrae
  ✓ 7. 4230-M-061Y1_jpg.rf.f65af9abb2c067343e5b11888dae948e.jpg - 7 vertebrae
  ✓ 8. Screenshot-2022-07-31-053514_jpg.rf.869dfa6a12bb66d90a757f98a2b1e0d6.jpg - 4 vertebrae
  ✓ 9. 3326-F-010Y1_jpg.rf.09484511d2529f1465a329351e0b3b10.jpg - 7 vertebrae
  ✓ 10. 3327-M-024Y1_jpg.rf.47bd1afb51097897051d95d52501ff4d.jpg - 9 vertebrae

✓ Saved visualization: /Users/mnourdine/phd/spondylolisthesis-maht-net/docs/figures/10_random_samples.png


/var/folders/rp/s8pcnsl15jzbb0w87z166p3w0000gn/T/ipykernel_72644/1538269923.py:56: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


## 15. Analyze Image Dimensions

Let's check the image dimensions to understand size variability.

In [59]:
# Collect image dimensions
image_dimensions = []
aspect_ratios = []

print("Analyzing image dimensions...\n")

for img_path in tqdm(train_image_files[:100], desc="Sampling 100 images"):  # Sample to avoid long processing
    try:
        img = cv2.imread(str(img_path))
        if img is not None:
            h, w = img.shape[:2]
            image_dimensions.append((w, h))
            aspect_ratios.append(w / h)
    except:
        pass

# Convert to DataFrame for analysis
dims_df = pd.DataFrame(image_dimensions, columns=['Width', 'Height'])

print("Image Dimension Statistics:")
print("=" * 60)
print(dims_df.describe())

print(f"\nAspect Ratio:")
print(f"  Min:    {min(aspect_ratios):.3f}")
print(f"  Max:    {max(aspect_ratios):.3f}")
print(f"  Mean:   {np.mean(aspect_ratios):.3f}")
print(f"  Median: {np.median(aspect_ratios):.3f}")

# Most common dimensions
print(f"\nMost Common Image Sizes:")
size_counts = dims_df.value_counts().head(5)
for (w, h), count in size_counts.items():
    print(f"  {w}×{h}: {count} images")

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Width distribution
axes[0].hist(dims_df['Width'], bins=20, color='skyblue', edgecolor='black', alpha=0.7)
axes[0].axvline(dims_df['Width'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {dims_df["Width"].mean():.0f}')
axes[0].set_xlabel('Width (pixels)', fontsize=12)
axes[0].set_ylabel('Count', fontsize=12)
axes[0].set_title('Image Width Distribution', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Height distribution
axes[1].hist(dims_df['Height'], bins=20, color='lightcoral', edgecolor='black', alpha=0.7)
axes[1].axvline(dims_df['Height'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {dims_df["Height"].mean():.0f}')
axes[1].set_xlabel('Height (pixels)', fontsize=12)
axes[1].set_ylabel('Count', fontsize=12)
axes[1].set_title('Image Height Distribution', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Aspect ratio distribution
axes[2].hist(aspect_ratios, bins=20, color='lightgreen', edgecolor='black', alpha=0.7)
axes[2].axvline(np.mean(aspect_ratios), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(aspect_ratios):.3f}')
axes[2].set_xlabel('Aspect Ratio (W/H)', fontsize=12)
axes[2].set_ylabel('Count', fontsize=12)
axes[2].set_title('Aspect Ratio Distribution', fontsize=14, fontweight='bold')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'image_dimensions.png', dpi=150, bbox_inches='tight')
print(f"\n✓ Saved: {FIGURES_DIR / 'image_dimensions.png'}")
plt.show()

Analyzing image dimensions...



Sampling 100 images:   0%|          | 0/100 [00:00<?, ?it/s]

Image Dimension Statistics:
       Width  Height
count  100.0   100.0
mean   640.0   640.0
std      0.0     0.0
min    640.0   640.0
25%    640.0   640.0
50%    640.0   640.0
75%    640.0   640.0
max    640.0   640.0

Aspect Ratio:
  Min:    1.000
  Max:    1.000
  Mean:   1.000
  Median: 1.000

Most Common Image Sizes:
  640×640: 100 images

✓ Saved: /Users/mnourdine/phd/spondylolisthesis-maht-net/docs/figures/image_dimensions.png


/var/folders/rp/s8pcnsl15jzbb0w87z166p3w0000gn/T/ipykernel_72644/1251181496.py:69: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


## 16. Phase 1.2 Summary

Let's create a summary of our visual exploration findings.

---

## Phase 1.2: Visual Exploration Summary

✅ **Completed Tasks**:
1. ✓ Selected 10 random training images
2. ✓ Created visualization function with color-coded vertebrae
3. ✓ Plotted annotated images in a 2×5 grid
4. ✓ Analyzed image dimensions and aspect ratios
5. ✓ Saved all visualizations to `docs/figures/`

**Key Observations**:
- **Annotation Quality**: Keypoints are accurately placed on vertebra corners
- **Color Coding**: L3(Red), L4(Green), L5(Blue), S1(Yellow) clearly visible
- **Vertebrae Detection**: Most images show all 4 vertebrae (L3, L4, L5, S1)
- **Image Variability**: Some variation in contrast, patient positioning, and vertebra sizes
- **Bounding Boxes**: Tightly fit around each vertebra

**Visual Quality Assessment**:
- Clear vertebrae visibility in most images
- Keypoints form proper rectangular shapes
- Annotations align well with anatomical structures
- Some images show varying levels of contrast

**Files Created**:
- `docs/figures/10_random_samples.png` - Grid of 10 annotated images
- `docs/figures/image_dimensions.png` - Dimension analysis plots

**Next Steps**: 
→ Phase 1.3: Statistical Analysis (keypoint distributions, slip percentages, clinical parameters)

---

## Phase 1.3: Statistical Analysis

Objective: compute per-vertebra statistics, inter-vertebral distances and a simple slip metric; produce summary tables and plots for clinical inspection.

Planned steps:
1. Build a per-vertebra table (boxes, centroids, keypoint means).
2. Compute slip percentage between adjacent vertebrae (simple proxy).
3. Plot distributions (box sizes, centroid positions, slip histogram).
4. Save CSVs and plots to `docs/statistics/` and `docs/figures/`.

In [60]:
# Phase 1.3 implementation: per-vertebra stats, slip metric, plots
import math

# Build per-vertebra table
rows = []
print('Building per-vertebra table...')
for ann in tqdm(train_annotations + val_annotations, desc='Images'):
    filename = ann.get('filename')
    boxes = ann.get('boxes', [])
    keypoints = ann.get('keypoints', [])
    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = [float(v) for v in box]
        w = x2 - x1
        h = y2 - y1
        area = w * h
        centroid_x = x1 + w/2.0
        centroid_y = y1 + h/2.0
        kp_mean_x = float('nan')
        kp_mean_y = float('nan')
        n_kp = 0
        if i < len(keypoints):
            kps = keypoints[i]
            xs = []
            ys = []
            vis_count = 0
            for kp in kps:
                try:
                    xk, yk, vk = kp
                    xs.append(float(xk))
                    ys.append(float(yk))
                    vis_count += 1 if int(vk) == 1 else 0
                except Exception:
                    continue
            if len(xs) > 0:
                kp_mean_x = float(np.mean(xs))
                kp_mean_y = float(np.mean(ys))
                n_kp = len(xs)
        rows.append({
            'filename': filename,
            'split': 'train' if ann in train_annotations else 'val',
            'vertebra_idx': i,
            'box_x1': x1,
            'box_y1': y1,
            'box_x2': x2,
            'box_y2': y2,
            'box_w': w,
            'box_h': h,
            'box_area': area,
            'centroid_x': centroid_x,
            'centroid_y': centroid_y,
            'kp_mean_x': kp_mean_x,
            'kp_mean_y': kp_mean_y,
            'n_keypoints': n_kp
        })

per_vert_df = pd.DataFrame(rows)
print(f'Per-vertebra rows: {len(per_vert_df)}')

# Save per-vertebra stats
per_vert_csv = STATS_DIR / 'per_vertebra_stats.csv'
per_vert_df.to_csv(per_vert_csv, index=False)
print(f"✓ Saved: {per_vert_csv}")

# Compute number of vertebrae per image
num_verts = per_vert_df.groupby('filename').size().rename('n_vertebrae').reset_index()
num_verts_path = STATS_DIR / 'vertebrae_counts_per_image.csv'
num_verts.to_csv(num_verts_path, index=False)
print(f"✓ Saved vertebra counts: {num_verts_path}")

# Compute slip metric between adjacent vertebrae per image
slip_rows = []
for filename, group in tqdm(per_vert_df.groupby('filename'), desc='Slip'):    
    group_sorted = group.sort_values('vertebra_idx').reset_index(drop=True)
    for j in range(len(group_sorted)-1):
        sup = group_sorted.loc[j]
        inf = group_sorted.loc[j+1]
        # slip proxy: horizontal displacement of superior centroid relative to inferior centroid
        # normalized by inferior box width
        try:
            dx = sup['centroid_x'] - inf['centroid_x']
            denom = inf['box_w'] if inf['box_w'] != 0 else np.nan
            slip_pct = (dx / denom) * 100.0 if not math.isnan(denom) else np.nan
        except Exception:
            slip_pct = np.nan
        slip_rows.append({
            'filename': filename,
            'sup_idx': int(sup['vertebra_idx']),
            'inf_idx': int(inf['vertebra_idx']),
            'dx': dx,
            'inf_box_w': inf['box_w'],
            'slip_pct': slip_pct
        })

slip_df = pd.DataFrame(slip_rows)
slip_csv = STATS_DIR / 'slip_metrics.csv'
slip_df.to_csv(slip_csv, index=False)
print(f"✓ Saved slip metrics: {slip_csv}")

# Quick summary: top slips
top_slips = slip_df.dropna(subset=['slip_pct']).reindex(slip_df['slip_pct'].abs().sort_values(ascending=False).index)
# take top 20 absolute slips
top20 = top_slips.head(20)
print('\nTop 10 absolute slip percentages (proxy):')
print(top20[['filename','sup_idx','inf_idx','slip_pct']].head(10).to_string(index=False))

# Save top offenders
top20.to_csv(STATS_DIR / 'top_slip_offenders.csv', index=False)
print(f"✓ Saved top offenders: {STATS_DIR / 'top_slip_offenders.csv'}")

# Mark Phase1.3 todo progress


Building per-vertebra table...


Images:   0%|          | 0/698 [00:00<?, ?it/s]

Per-vertebra rows: 4600
✓ Saved: /Users/mnourdine/phd/spondylolisthesis-maht-net/docs/statistics/per_vertebra_stats.csv
✓ Saved vertebra counts: /Users/mnourdine/phd/spondylolisthesis-maht-net/docs/statistics/vertebrae_counts_per_image.csv


Slip:   0%|          | 0/698 [00:00<?, ?it/s]

✓ Saved slip metrics: /Users/mnourdine/phd/spondylolisthesis-maht-net/docs/statistics/slip_metrics.csv

Top 10 absolute slip percentages (proxy):
                                                                 filename  sup_idx  inf_idx    slip_pct
                3953-M-021Y1_jpg.rf.e4f27876fbd73af3dc3bfaf925c19fa5.json        6        7 1252.777778
Screenshot-2022-07-31-051032_jpg.rf.e815841d374954ac9690d2e8939440ad.json        3        4  111.734694
Screenshot-2022-07-31-053718_jpg.rf.e062a3998063b692f6124b1a6c2e86f7.json        3        4  103.645833
                3255-F-061Y1_jpg.rf.cdf702fcb7c71085eef28d4b3988f73e.json        4        5 -101.960784
                3715-F-022Y1_jpg.rf.d4a008fcd237a4a554b1b438d0e3745f.json        5        6 -101.041667
Screenshot-2022-07-31-053718_jpg.rf.c5a8ffe1da922f55dcd640ac64ced680.json        3        4 -101.020408
 spondylolithesis_jumbo_jpeg_jpg.rf.073dd7a24bfabf1bc52fff2719006a44.json        5        6  -98.148148
Screenshot-2022-07-31-

In [61]:
# Create slip distribution histogram
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Filter out extreme outliers for better visualization
slip_values = slip_df['slip_pct'].dropna()
q1, q99 = slip_values.quantile([0.01, 0.99])

# Full distribution
ax1 = axes[0]
ax1.hist(slip_values, bins=50, edgecolor='black', alpha=0.7, color='skyblue')
ax1.axvline(0, color='red', linestyle='--', linewidth=2, label='No slip (0%)')
ax1.axvline(slip_values.mean(), color='green', linestyle='--', linewidth=2, 
           label=f'Mean: {slip_values.mean():.1f}%')
ax1.axvline(slip_values.median(), color='orange', linestyle='--', linewidth=2, 
           label=f'Median: {slip_values.median():.1f}%')
ax1.set_xlabel('Slip Percentage (%)', fontsize=12)
ax1.set_ylabel('Count', fontsize=12)
ax1.set_title('Slip Distribution - Full Range', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Zoomed distribution (1st to 99th percentile)
ax2 = axes[1]
slip_filtered = slip_values[(slip_values >= q1) & (slip_values <= q99)]
ax2.hist(slip_filtered, bins=50, edgecolor='black', alpha=0.7, color='lightcoral')
ax2.axvline(0, color='red', linestyle='--', linewidth=2, label='No slip (0%)')
ax2.axvline(slip_filtered.mean(), color='green', linestyle='--', linewidth=2, 
           label=f'Mean: {slip_filtered.mean():.1f}%')
ax2.axvline(slip_filtered.median(), color='orange', linestyle='--', linewidth=2, 
           label=f'Median: {slip_filtered.median():.1f}%')
ax2.set_xlabel('Slip Percentage (%)', fontsize=12)
ax2.set_ylabel('Count', fontsize=12)
ax2.set_title('Slip Distribution - Central 98% (1st-99th percentile)', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()

# Save figure
output_path = FIGURES_DIR / 'slip_distribution.png'
plt.savefig(output_path, dpi=150, bbox_inches='tight')
print(f"\n✓ Saved slip distribution: {output_path}")

# Print statistics
print("\nSlip Statistics:")
print("=" * 60)
print(f"Total slip measurements: {len(slip_values)}")
print(f"Mean slip:     {slip_values.mean():.2f}%")
print(f"Median slip:   {slip_values.median():.2f}%")
print(f"Std dev:       {slip_values.std():.2f}%")
print(f"Min slip:      {slip_values.min():.2f}%")
print(f"Max slip:      {slip_values.max():.2f}%")
print(f"\nPercentiles:")
print(f"  1st:  {slip_values.quantile(0.01):.2f}%")
print(f"  5th:  {slip_values.quantile(0.05):.2f}%")
print(f"  25th: {slip_values.quantile(0.25):.2f}%")
print(f"  50th: {slip_values.quantile(0.50):.2f}%")
print(f"  75th: {slip_values.quantile(0.75):.2f}%")
print(f"  95th: {slip_values.quantile(0.95):.2f}%")
print(f"  99th: {slip_values.quantile(0.99):.2f}%")

# Count severe slips (arbitrary threshold of >50% or <-50%)
severe_forward = len(slip_values[slip_values > 50])
severe_backward = len(slip_values[slip_values < -50])
print(f"\nSevere Slips (>50% displacement):")
print(f"  Forward (anterolisthesis):  {severe_forward} cases ({severe_forward/len(slip_values)*100:.1f}%)")
print(f"  Backward (retrolisthesis):  {severe_backward} cases ({severe_backward/len(slip_values)*100:.1f}%)")

plt.show()


✓ Saved slip distribution: /Users/mnourdine/phd/spondylolisthesis-maht-net/docs/figures/slip_distribution.png

Slip Statistics:
Total slip measurements: 3902
Mean slip:     1.12%
Median slip:   5.18%
Std dev:       32.85%
Min slip:      -101.96%
Max slip:      1252.78%

Percentiles:
  1st:  -66.66%
  5th:  -47.78%
  25th: -14.89%
  50th: 5.18%
  75th: 18.67%
  95th: 35.13%
  99th: 57.62%

Severe Slips (>50% displacement):
  Forward (anterolisthesis):  62 cases (1.6%)
  Backward (retrolisthesis):  157 cases (4.0%)


/var/folders/rp/s8pcnsl15jzbb0w87z166p3w0000gn/T/ipykernel_72644/1949030841.py:69: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


## 17. Visualize Top Slip Cases

Let's visualize the images with the highest slip percentages to understand what large slip values represent anatomically.

In [62]:
# Visualize top 6 slip cases (3x2 grid)
n_cases = 6
fig, axes = plt.subplots(2, 3, figsize=(20, 14), gridspec_kw={'hspace': 0.35, 'wspace': 0.2})
axes = axes.flatten()

print(f"Visualizing top {n_cases} slip cases...\n")

for idx in range(n_cases):
    if idx >= len(top20):
        axes[idx].axis('off')
        continue
    
    row = top20.iloc[idx]
    filename = row['filename']
    sup_idx = int(row['sup_idx'])
    inf_idx = int(row['inf_idx'])
    slip_pct = row['slip_pct']
    
    # Find image file (try different extensions)
    img_stem = filename.replace('.json', '').replace('_jpg.rf.', '_jpg.rf.')
    img_path = None
    
    # Search in both train and val directories
    for search_dir in [TRAIN_IMAGES_DIR, VAL_IMAGES_DIR]:
        for ext in ['.jpg', '.png', '.jpeg']:
            # Try exact match first
            candidate = search_dir / filename.replace('.json', ext)
            if candidate.exists():
                img_path = candidate
                break
            # Try with _jpg pattern
            candidate = search_dir / filename.replace('.json', '').replace('_jpg.rf.', '_jpg.rf.').split('_jpg.rf.')[0]
            for possible_file in search_dir.glob(f"{candidate.stem}*{ext}"):
                img_path = possible_file
                break
        if img_path:
            break
    
    if not img_path or not img_path.exists():
        # Try a more aggressive search
        base_search = filename.split('_jpg.rf.')[0] if '_jpg.rf.' in filename else filename.replace('.json', '')
        for search_dir in [TRAIN_IMAGES_DIR, VAL_IMAGES_DIR]:
            matches = list(search_dir.glob(f"{base_search}*"))
            if matches:
                img_path = matches[0]
                break
    
    ax = axes[idx]
    
    if img_path and img_path.exists():
        try:
            # Load image
            image = cv2.imread(str(img_path))
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Find annotation
            json_path = None
            for label_dir in [TRAIN_LABELS_DIR, VAL_LABELS_DIR]:
                candidate = label_dir / filename
                if candidate.exists():
                    json_path = candidate
                    break
            
            if json_path:
                with open(json_path, 'r') as f:
                    annotation = json.load(f)
                
                # Draw all annotations
                annotated = draw_annotations(image_rgb, annotation)
                
                # Highlight the specific vertebrae pair with the slip
                boxes = annotation['boxes']
                if sup_idx < len(boxes) and inf_idx < len(boxes):
                    sup_box = boxes[sup_idx]
                    inf_box = boxes[inf_idx]
                    
                    # Draw special markers for the slip pair
                    # Superior vertebra - thick red outline
                    x1, y1, x2, y2 = map(int, sup_box)
                    cv2.rectangle(annotated, (x1, y1), (x2, y2), (255, 0, 0), 4)
                    
                    # Inferior vertebra - thick orange outline
                    x1, y1, x2, y2 = map(int, inf_box)
                    cv2.rectangle(annotated, (x1, y1), (x2, y2), (255, 165, 0), 4)
                    
                    # Draw horizontal line showing displacement
                    sup_cx = (sup_box[0] + sup_box[2]) / 2
                    sup_cy = (sup_box[1] + sup_box[3]) / 2
                    inf_cx = (inf_box[0] + inf_box[2]) / 2
                    inf_cy = (inf_box[1] + inf_box[3]) / 2
                    
                    cv2.arrowedLine(annotated, 
                                   (int(inf_cx), int(inf_cy)), 
                                   (int(sup_cx), int(inf_cy)),
                                   (255, 255, 0), 3, tipLength=0.05)
                
                ax.imshow(annotated)
                
                # Create title with slip info
                direction = "→" if slip_pct > 0 else "←"
                title = f"#{idx+1}: Slip = {abs(slip_pct):.1f}% {direction}\n"
                title += f"V{sup_idx} → V{inf_idx} | {filename[:40]}"
                ax.set_title(title, fontsize=10, fontweight='bold', pad=5)
                
                print(f"  ✓ {idx+1}. {filename[:50]} - Slip: {slip_pct:.1f}% (V{sup_idx}→V{inf_idx})")
            else:
                ax.text(0.5, 0.5, f'Annotation not found\n{filename[:30]}', 
                       ha='center', va='center', transform=ax.transAxes)
                print(f"  ✗ {idx+1}. Annotation not found: {filename}")
        
        except Exception as e:
            ax.text(0.5, 0.5, f'Error loading\n{str(e)[:50]}', 
                   ha='center', va='center', transform=ax.transAxes)
            print(f"  ✗ {idx+1}. Error: {e}")
    else:
        ax.text(0.5, 0.5, f'Image not found\n{filename[:30]}', 
               ha='center', va='center', transform=ax.transAxes)
        print(f"  ✗ {idx+1}. Image not found: {filename}")
    
    ax.axis('off')

plt.suptitle('Top Slip Cases - Visual Inspection\n' + 
             'Red outline: Superior vertebra | Orange outline: Inferior vertebra | Yellow arrow: Horizontal displacement',
             fontsize=14, fontweight='bold', y=0.99)
plt.tight_layout(rect=[0, 0, 1, 0.97])

# Save figure
output_path = FIGURES_DIR / 'top_slip_cases.png'
plt.savefig(output_path, dpi=150, bbox_inches='tight')
print(f"\n✓ Saved visualization: {output_path}")

plt.show()

Visualizing top 6 slip cases...

  ✓ 1. 3953-M-021Y1_jpg.rf.e4f27876fbd73af3dc3bfaf925c19f - Slip: 1252.8% (V6→V7)
  ✓ 2. Screenshot-2022-07-31-051032_jpg.rf.e815841d374954 - Slip: 111.7% (V3→V4)
  ✓ 3. Screenshot-2022-07-31-053718_jpg.rf.e062a3998063b6 - Slip: 103.6% (V3→V4)
  ✓ 4. 3255-F-061Y1_jpg.rf.cdf702fcb7c71085eef28d4b3988f7 - Slip: -102.0% (V4→V5)
  ✓ 5. 3715-F-022Y1_jpg.rf.d4a008fcd237a4a554b1b438d0e374 - Slip: -101.0% (V5→V6)
  ✓ 6. Screenshot-2022-07-31-053718_jpg.rf.c5a8ffe1da922f - Slip: -101.0% (V3→V4)


/var/folders/rp/s8pcnsl15jzbb0w87z166p3w0000gn/T/ipykernel_72644/1221695738.py:125: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 1, 0.97])



✓ Saved visualization: /Users/mnourdine/phd/spondylolisthesis-maht-net/docs/figures/top_slip_cases.png


/var/folders/rp/s8pcnsl15jzbb0w87z166p3w0000gn/T/ipykernel_72644/1221695738.py:132: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


## 18. Side-by-Side Comparison: Normal vs Severe Slip

Let's compare a case with minimal slip (normal alignment) against the worst slip case to clearly understand the displacement.

### Key Findings from the Visualizations:
#### Top Slip Cases (Image Grid):
- Case #1 (1252.8% slip) - This is an extreme outlier with a very small vertebra (V8) that appears to be displaced far to the left. The yellow arrow shows the massive horizontal displacement.

- Cases #2 & #3 (~100-111% slip) - Show vertebrae where the superior vertebra is displaced forward (anteriorly) by about the width of the inferior vertebra - classic anterolisthesis.

- Cases #4, #5, #6 (~-101% slip) - Negative values indicate backward displacement (retrolisthesis), where the superior vertebra is shifted posteriorly relative to the inferior one.

#### Visual Markers:
- Red thick outline: Superior vertebra (the one that's displaced)
- Orange thick outline: Inferior vertebra (reference point)
- Yellow arrow: Shows the horizontal displacement between the two vertebrae centroids

In [63]:
# Find a case with minimal slip (close to 0%)
normal_case = slip_df[slip_df['slip_pct'].abs() < 5].iloc[0] if len(slip_df[slip_df['slip_pct'].abs() < 5]) > 0 else slip_df.iloc[len(slip_df)//2]

# Randomly select one of the worst slip cases from top 20
np.random.seed()  # Use current time for randomness
random_worst_idx = np.random.randint(0, min(20, len(top20)))
worst_case = top20.iloc[random_worst_idx]
print(f"Randomly selected case #{random_worst_idx + 1} from top 20 worst slips\n")

# Create side-by-side visualization
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

cases = [
    (normal_case, "Normal Alignment", axes[0]),
    (worst_case, "Severe Slip", axes[1])
]

print("Comparing Normal vs Severe Slip Cases:\n")

for case, case_type, ax in cases:
    filename = case['filename']
    sup_idx = int(case['sup_idx'])
    inf_idx = int(case['inf_idx'])
    slip_pct = case['slip_pct']
    
    # Find image file
    img_path = None
    for search_dir in [TRAIN_IMAGES_DIR, VAL_IMAGES_DIR]:
        for ext in ['.jpg', '.png', '.jpeg']:
            candidate = search_dir / filename.replace('.json', ext)
            if candidate.exists():
                img_path = candidate
                break
            base_search = filename.split('_jpg.rf.')[0] if '_jpg.rf.' in filename else filename.replace('.json', '')
            matches = list(search_dir.glob(f"{base_search}*"))
            if matches:
                img_path = matches[0]
                break
        if img_path:
            break
    
    if img_path and img_path.exists():
        try:
            # Load image
            image = cv2.imread(str(img_path))
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Find annotation
            json_path = None
            for label_dir in [TRAIN_LABELS_DIR, VAL_LABELS_DIR]:
                candidate = label_dir / filename
                if candidate.exists():
                    json_path = candidate
                    break
            
            if json_path:
                with open(json_path, 'r') as f:
                    annotation = json.load(f)
                
                # Draw all annotations
                annotated = draw_annotations(image_rgb, annotation)
                
                # Highlight the specific vertebrae pair
                boxes = annotation['boxes']
                if sup_idx < len(boxes) and inf_idx < len(boxes):
                    sup_box = boxes[sup_idx]
                    inf_box = boxes[inf_idx]
                    
                    # Superior vertebra - thick red outline
                    x1, y1, x2, y2 = map(int, sup_box)
                    cv2.rectangle(annotated, (x1, y1), (x2, y2), (255, 0, 0), 5)
                    
                    # Inferior vertebra - thick orange outline
                    x1, y1, x2, y2 = map(int, inf_box)
                    cv2.rectangle(annotated, (x1, y1), (x2, y2), (255, 165, 0), 5)
                    
                    # Calculate centroids
                    sup_cx = (sup_box[0] + sup_box[2]) / 2
                    sup_cy = (sup_box[1] + sup_box[3]) / 2
                    inf_cx = (inf_box[0] + inf_box[2]) / 2
                    inf_cy = (inf_box[1] + inf_box[3]) / 2
                    
                    # Draw horizontal displacement arrow
                    cv2.arrowedLine(annotated, 
                                   (int(inf_cx), int(inf_cy)), 
                                   (int(sup_cx), int(inf_cy)),
                                   (0, 255, 255), 4, tipLength=0.05)
                    
                    # Draw vertical reference lines from centroids
                    img_height = annotated.shape[0]
                    cv2.line(annotated, (int(inf_cx), 0), (int(inf_cx), img_height), 
                            (255, 165, 0), 2, cv2.LINE_AA)
                    cv2.line(annotated, (int(sup_cx), 0), (int(sup_cx), img_height), 
                            (255, 0, 0), 2, cv2.LINE_AA)
                    
                    # Add text annotations
                    cv2.putText(annotated, f"Slip: {slip_pct:.1f}%", 
                               (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)
                    cv2.putText(annotated, f"Superior V{sup_idx}", 
                               (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
                    cv2.putText(annotated, f"Inferior V{inf_idx}", 
                               (10, 115), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 165, 0), 2)
                
                ax.imshow(annotated)
                
                # Title
                direction = "→ Forward" if slip_pct > 0 else "← Backward" if slip_pct < 0 else "Aligned"
                title = f"{case_type}\n"
                title += f"Slip: {abs(slip_pct):.1f}% {direction}\n"
                title += f"V{sup_idx} → V{inf_idx}"
                ax.set_title(title, fontsize=14, fontweight='bold', pad=15)
                
                print(f"{case_type}:")
                print(f"  File: {filename[:60]}")
                print(f"  Slip: {slip_pct:.1f}% (V{sup_idx}→V{inf_idx})")
                print(f"  Displacement: {case.get('dx', 0):.1f} pixels")
                print()
                
        except Exception as e:
            ax.text(0.5, 0.5, f'Error: {str(e)[:50]}', 
                   ha='center', va='center', transform=ax.transAxes)
            print(f"  Error loading {case_type}: {e}")
    else:
        ax.text(0.5, 0.5, f'Image not found', 
               ha='center', va='center', transform=ax.transAxes)
        print(f"  Image not found for {case_type}")
    
    ax.axis('off')

plt.suptitle('Comparison: Normal Alignment vs Severe Vertebral Slip\n' + 
             'Red outline: Superior vertebra | Orange outline: Inferior vertebra | Yellow arrow: Horizontal displacement | Vertical lines: Centroid alignment',
             fontsize=16, fontweight='bold', y=0.98)
plt.tight_layout(rect=[0, 0, 1, 0.95])

# Save figure
output_path = FIGURES_DIR / 'normal_vs_severe_slip_comparison.png'
plt.savefig(output_path, dpi=150, bbox_inches='tight')
print(f"✓ Saved comparison: {output_path}")

plt.show()

Randomly selected case #1 from top 20 worst slips

Comparing Normal vs Severe Slip Cases:

Normal Alignment:
  File: 2695-F-026Y1_jpg.rf.3df07db3271aac57004609a856919a96.json
  Slip: 0.6% (V2→V3)
  Displacement: 0.5 pixels

Severe Slip:
  File: 3953-M-021Y1_jpg.rf.e4f27876fbd73af3dc3bfaf925c19fa5.json
  Slip: 1252.8% (V6→V7)
  Displacement: 225.5 pixels

✓ Saved comparison: /Users/mnourdine/phd/spondylolisthesis-maht-net/docs/figures/normal_vs_severe_slip_comparison.png


/var/folders/rp/s8pcnsl15jzbb0w87z166p3w0000gn/T/ipykernel_72644/1714874663.py:140: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
